# HW3 - Q3 [35 pts]

## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> add any cells to this Jupyter Notebook, because that will crash the autograder.
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> implement any additional libraries into this workbook.
</div>

All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  

<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`bucket()`

`long_trips()`

`manhattan_trips()`

`weighted_profit()`

`final_output()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from the NYC-TLC dataset during auto-grading.

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove or modify the following utility functions:
</div>

`load_data()`

`main()`

<div class="alert alert-block alert-info">
    Do <strong>not</strong> change the below cell. Run it to initialize your PySpark instance. If you don't get any output, make sure your Notebook's Kernel is set to "PySpark" in the top right corner.
</div>

In [1]:
sc

''

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remodify the below cell. It contains the function for loading data and all imports, and the function for running your code.
</div>

In [2]:
#### DO NOT CHANGE ANYTHING IN THIS CELL ####

from pyspark.sql.functions import col
from pyspark.sql import *
from pyspark.sql import functions as F

def load_data(size='small'):
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'
    
    if size != 'small' and size != 'large':
        print("Invalid size parameter provided. Use only 'small' or 'large'.")
        return
    
    input_bucket = "s3://cse6242-spring2021"
    
    # Load Trip Data
    trip_path = '/'+size+'/yellow_tripdata*'
    trips = sqlContext.read.option("header",True).csv("yellow_tripdata_2019-01_short.csv")
    print("Trip Count: ",trips.count()) # Prints # of trips (# of records, as each record is one trip)
    
    # Load Lookup Data
    lookup_path = '/'+size+'/taxi*'
    lookup = sqlContext.read.option("header",True).csv("taxi_zone_lookup.csv")
    
    return trips, lookup

def main(size, bucket):
    # Runs your functions implemented above.
    
    print(user())
    trips, lookup = load_data(size=size)
    trips = long_trips(trips)
    mtrips = manhattan_trips(trips, lookup)
    wp = weighted_profit(trips, mtrips)
    print("flag-----:")
    lookup.show()
    final = final_output(wp,lookup)
    
    # Outputs the results for you to visually see
    final.show()
    
    # Writes out as a CSV to your bucket.
    #final.write.csv(bucket)

# Implement the below functions for this assignment:
<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> change any function inputs or outputs, and ensure that the dataframes your code returns align with the schema definitions commented in each function
</div>

## 3a. [1 pt] Update the `user()` function
This function should return your GT username, eg: gburdell3

In [3]:
def user():
    # Returns a string consisting of your GT username.
    return 'zpatwary3'

## 3b. [2 pts] Update the `long_trips()` function
This function filters trips to keep only trips greater than or equal to 2 miles.

In [4]:
def long_trips(trips):
    # Returns a Dataframe with Schema the same as :trips:
    trips = trips.filter(trips.trip_distance >=2)
    return trips

## 3c. [6 pts] Update the `manhattan_trips()` function

This function determines the top 20 locations with a `DOLocationID` in manhattan by passenger_count (pcount).

Example output formatting:

```
+--------------+--------+
| DOLocationID | pcount |
+--------------+--------+
|             5|      15|
|            16|      12| 
+--------------+--------+
```

In [5]:
def manhattan_trips(trips, lookup):
    # Returns a Dataframe with Schema: DOLocationID, pcount
    man_lookup = lookup.filter(lookup.Borough=="Manhattan")
    man_lookup_list = [row[0] for row in lookup.select('LocationId').collect()]
    
    man_trips = trips.where(col("DOLocationID").isin(man_lookup_list))
    man_trips = man_trips.select("DOLocationID", "passenger_count")      
    man_trips = man_trips.groupBy("DOLocationID")
    man_trips = man_trips.agg({'passenger_count':'sum'}).orderBy(col("sum(passenger_count)").desc()).withColumnRenamed("sum(passenger_count)", "pcount")
    #man_trips.show(20)
    return man_trips

## 3d. [6 pts] Update the `weighted_profit()` function
This function should determine the average `total_amount`, the total count of trips, and the total count of trips ending in the top 20 destinations and return the `weighted_profit` as discussed in the homework document.

Example output formatting:
```
+--------------+-------------------+
| PULocationID |  weighted_profit  |
+--------------+-------------------+
|            18| 33.784444421924436| 
|            12| 21.124577637149223| 
+--------------+-------------------+
```

In [6]:
def weighted_profit(trips, mtrips): 
    # Returns a Dataframe with Schema: PULocationID, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    
    #trips.printSchema()
    #mtrips.printSchema()
    #avg_total_amount = trips.agg({'total_amount':'avg'}).withColumnRenamed("avg(total_amount)", "avg_total_amount")
    #avg_total_amount = trips.groupBy("PULocationID").agg(F.mean('total_amount')).show(5)   #.withColumnRenamed("avg(total_amount)", "avg_total_amount")
    avg_total_amount = trips.groupBy("PULocationID").avg( "total_amount").withColumnRenamed("avg(total_amount)", "avg_total_amount")
    #avg_total_amount.show(5)
    
    #    total count of trips from a PULocationID
    trip_count_puloc = trips.groupBy("PULocationID").count().withColumnRenamed("count", "count_trips")
    #trip_count_puloc.show(10)
    
    #   total count of trips ending in the top 20 destinations
    top20_doloc = mtrips.limit(20)
    #print(top20_trips.count())
    top20_doloc_list = [row[0] for row in top20_doloc.select('DOLocationId').collect()]
    top20_trips = trips.where(col("DOLocationID").isin(top20_doloc_list))
    #top20_trips.show()
    trip_count_puloc_top20 = top20_trips.groupBy("PULocationID").count().withColumnRenamed("count", "count_top20")
    #trip_count_puloc_top20.show(10)
    
    # Calculate proportion
    # df004 = df003.as('a).join(trip_count_puloc_top20.as('b), $"a.PULocationID" === $"b.PULocationID","left").select(trip_count_puloc("Borough"), df003("number_activities").alias("total_number_activities")).na.fill(0)
    df_prop = trip_count_puloc.join(trip_count_puloc_top20, "PULocationID") \
            .withColumn("proportion", (col("count_top20") / col("count_trips"))) \
            .drop("count_trips", "count_top20")
    #df_prop.show(5)
       
    # Calculate weighted_profit
    weighted_profit = df_prop.join(avg_total_amount, "PULocationID") \
            .withColumn("weighted_profit", (col("avg_total_amount") * col("proportion"))) \
            .drop("avg_total_amount", "proportion")
    weighted_profit.show(5)
    return weighted_profit
    

## 3e. [5 pts] Update the `final_output()` function
This function will take the results of `weighted_profit`, links it to the `borough` and `zone` and returns the top 20 locations with the highest `weighted_profit`.

Example output formatting:
```
+------------+---------+-------------------+
|    Zone    | Borough |  weighted_profit  |
+----------------------+-------------------+
| JFK Airport|   Queens|  16.95897820117925|
|     Jamaica|   Queens| 14.879835188762488|
+------------+---------+-------------------+
```

In [9]:
def final_output(wp, lookup): 
    # Returns a Dataframe with Schema: Zone, Borough, weighted_profit
    # Note: Use decimal datatype for weighted profit (NOTE: DON'T USE FLOAT)
    # Our grader will be only be checking the first 8 characters for each value in the dataframe
    
    # df004 = df003.as('a).join(trip_count_puloc_top20.as('b), $"a.PULocationID" === $"b.PULocationID","left").select(trip_count_puloc("Borough"), df003("number_activities").alias("total_number_activities")).na.fill(0)
    #df_final = wp.join(lookup, wp.PULocationID == lookup.LocationID,"left").select(lookup("Zone"), wp("weighted_profit")).na.fill(0) 
            #.drop("avg_total_amount", "proportion")
    
    lookup.show()
    #print(lookup)
    #df_final = wp.withColumn("id", col("PULocationID")).join(lookup.withColumn("id", col("LocationID")), on="id").select("lookup.Zone", "lookup.Borough", "wp.weighted_profit")
    #df_final.show(5)
    #return df_final
    wp.join(lookup, lookup.PULocationID == df2.LocationID, 'outer')
    wp.show(5)
    pass

<div class="alert alert-block alert-info">
    Test your code on the small dataset first, as the large dataset will take a significantly longer time to run
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> use the same bucket url for multiple runs of the `main()` function, as this will cause errors. Make sure to change the name of your output location every time. (ie: s3://cse6242-gburdell3/output-small2)
</div>

Update the below cell with the path to your bucket, then run the below cell to run your code to store the results in S3.

When you have confirmed the results of the small dataset, run it again using the large dataset. Your output file will appear in a folder in your s3 bucket called YOUROUTPUT.csv as a csv file with a name something like part-0000-4d992f7a-0ad3-48f8-8c72-0022984e4b50-c000.csv. Download this file and rename it to q3_output.csv for submission. Do not make any other changes to the file. 

In [10]:
# update your bucket path
bucket = 's3://cse6242-zpatwary3/output-small'
main('small', bucket)

# bucket = 's3://cse6242-zpatwary3/output-large'
# main('large', bucket)

zpatwary3


NameError: name 'sqlContext' is not defined

#### Testing

<div class="alert alert-block alert-info">
    You may use the below cell for any additional testing you need to do, however any code implemented below will not be run or used when grading
</div>